In [3]:
# importing libraries
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import time

In [4]:
# importing cleaned csv
df_cleaned = pd.read_csv('norm_hikes.csv')
# print(df_cleaned.head())

# Geocoding the locations
geolocator = Nominatim(
    user_agent="kenya-hiking-research-2026"
)

def geocode_osm(query):
    try:
        location = geolocator.geocode(
            query,
            country_codes="ke",
            timeout=10
        )
        if location:
            return location.latitude, location.longitude, location.address
    except (GeocoderTimedOut, GeocoderUnavailable):
        return None, None, None

    return None, None, None
    


In [5]:
# putting to columns
lats, lons, addresses = [], [], []

df_cleaned["query"] = (
    df_cleaned["starting_point"].str.strip()
)

for q in df_cleaned["query"]:
    lat, lon, addr = geocode_osm(q)
    lats.append(lat)
    lons.append(lon)
    addresses.append(addr)

    time.sleep(1)  

df_cleaned["lat"] = lats
df_cleaned["lon"] = lons
df_cleaned["osm_address"] = addresses


In [10]:
failed = df_cleaned[df_cleaned["lat"].isna()]
failed[["query"]].head()

# print(df_cleaned)
df_cleaned.to_csv("norm_geocoded.csv", index=False)
